In [2]:
import sys
import pennylane as qml
from pennylane import numpy as np

def find_subsets(l,v):
    """This function will determine subsets where the sum is equal to the value v in the following vector l
    Args:
        - l (list): input vector 
        - v (integer): input positive integer 
    output:
        - print the possible subsets 
    """
    n=len(l)
    m=5
    dev = qml.device("default.qubit", wires=n+m)
    def CU1(theta,bk,wires):
        qml.QubitUnitary(np.array([[1,0],[0,1-bk+(np.cos(theta)+1j*np.sin(theta))*bk]]),wires=wires)
    def add_part(i):
        b=[int(j) for j in np.binary_repr(l[i], width=m)]
        for j in range(m):
            for k in range(j,m):
                CU1(theta=2*np.pi/2**(k-j+1),bk=b[k],wires=n+m-(j+1)) 
    def add():
        qml.QFT(wires=range(n,n+m))   
        for i in range(n):
            qml.ctrl(add_part,control=i)(i)        
        qml.adjoint(qml.QFT)(wires=range(n,n+m))
    def oracle():
        add()
        arr=np.eye(2**m)
        arr[v,v]=-1
        qml.QubitUnitary(arr,wires=range(n,n+m))
        qml.adjoint(add)() 
    @qml.qnode(dev)     
    def find():
        qml.BasisState(np.array([0]*(n+m)),wires=range(n+m))
        for i in range(n):
            qml.Hadamard(wires=i)             
        for t in range(3):          
            oracle()
            for i in range(n):
                qml.Hadamard(wires=i)
            arr=np.eye(2**n)
            for i in range(1,2**n):
                arr[i,i]=-1
            qml.QubitUnitary(arr,wires=range(n))
            for i in range(n):
                qml.Hadamard(wires=i)
        return qml.probs(wires=range(n))
    res=find()
    subsets=[]
    for i in range(len(res)):
        if res[i]>np.max(res)-0.0001:
            li=[int(j) for j in np.binary_repr(i, width=n)] 
            subsets.append([l[j] for j in range(len(li)) if li[j]==1])
                   
                
    return subsets
if __name__ == "__main__":
    #Example l=[5,7,8,9,1],v=16
    print(find_subsets(l=[5,7,8,9,1],v=16))

[[7, 9], [7, 8, 1]]
